In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import sys
sys.path.append(str(Path.cwd() / "_scripts"))

In [3]:
from case_edits.ezcase_new import *
from helpers.ep_helpers import *
from helpers.helpers import *
from eppy.bunch_subclass import EpBunch
from airflow_network.logic import pair_zone_and_subsurfaces


In [4]:

inputs_dir = "case_red_d3"
outputs_dir = "tests/test22_svg2plan"

case = initialize_case(outputs_dir)
path_to_inputs = get_path_to_inputs(inputs_dir)
pairs = get_subsurface_pairs_from_case(path_to_inputs)




In [5]:
idf = create_ezcase(outputs_dir, inputs_dir)

width: 0.57 NOT GREATER THAN 0.81
Editing (2, 6) 

width: 0.66 NOT GREATER THAN 0.81
Editing (2, 7) 

width: 0.71 NOT GREATER THAN 0.81
Editing (6, 4) 

width: 0.71 NOT GREATER THAN 0.81
Editing (3, 6) 



In [7]:
idf.idfobjects["AIRFLOWNETWORK:MULTIZONE:SURFACE"]

[
AIRFLOWNETWORK:MULTIZONE:SURFACE,
    Block 01 Storey 0 Wall 0003 Window,    !- Surface Name
    Block 01 Storey 0 Wall 0003 Window SimpleOpening,    !- Leakage Component Name
    ,                         !- External Node Name
    1,                        !- WindowDoor Opening Factor or Crack Factor
    ZoneLevel,                !- Ventilation Control Mode
    ,                         !- Ventilation Control Zone Temperature Setpoint Schedule Name
    0,                        !- Minimum Venting Open Factor
    0,                        !- Indoor and Outdoor Temperature Difference Lower Limit For Maximum Venting Open Factor
    100,                      !- Indoor and Outdoor Temperature Difference Upper Limit for Minimum Venting Open Factor
    0,                        !- Indoor and Outdoor Enthalpy Difference Lower Limit For Maximum Venting Open Factor
    300000,                   !- Indoor and Outdoor Enthalpy Difference Upper Limit for Minimum Venting Open Factor
    ,        

In [10]:
subsurfaces = chain_flatten(pair_zone_and_subsurfaces(idf).values())
len(subsurfaces), len(set(subsurfaces))

AttributeError: 'NoneType' object has no attribute 'values'

In [38]:
res: list[EpBunch] = [i for i in get_zones(idf)]
res[0].values()

dict_values([['ZONE', 'Block 00 Storey 0', 0.0, 0.0, 0.0, 0.0, 1, 1, 'autocalculate', 'autocalculate', 'autocalculate', '', '', 'Yes'], ['key', 'Name', 'Direction_of_Relative_North', 'X_Origin', 'Y_Origin', 'Z_Origin', 'Type', 'Multiplier', 'Ceiling_Height', 'Volume', 'Floor_Area', 'Zone_Inside_Convection_Algorithm', 'Zone_Outside_Convection_Algorithm', 'Part_of_Total_Floor_Area'], [{'memo': ['Defines a thermal zone of the building. Every zone contains one or more Spaces.', 'Space is an optional input.', 'If a Zone has no Space(s) specified in input then a default Space named <Zone Name> will be created.', 'If some surfaces in a Zone are assigned to a space and some are not, then a default Space', 'named <Zone Name>-Remainder will be created.', 'Input references to Space Names must have a matching Space object', '(default space names may not be referenced except in output variable keys).'], 'format': ['vertices'], 'group': 'Thermal Zones and Surfaces', 'idfobj': 'Zone'}, {'field': ['Na

In [6]:
zone_names = [i.Name for i in get_zones(idf)]
zone_names

['Block 00 Storey 0',
 'Block 01 Storey 0',
 'Block 02 Storey 0',
 'Block 03 Storey 0',
 'Block 04 Storey 0',
 'Block 05 Storey 0',
 'Block 06 Storey 0',
 'Block 07 Storey 0']

In [7]:
def find_matching_zone(subsurface:EpBunch):
    for i in zone_names:
        if i in subsurface.Building_Surface_Name:
            return i

In [8]:
original_subsurfaces = [i for i in idf.getsubsurfaces() if "Partner" not in i.Name]
original_subsurfaces[0]


WINDOW,
    Block 04 Storey 0 Wall 0001 Window,    !- Name
    Project External Window,    !- Construction Name
    Block 04 Storey 0 Wall 0001,    !- Building Surface Name
    ,                         !- Frame and Divider Name
    1.0,                      !- Multiplier
    2.12,                     !- Starting X Coordinate
    1.4994999999999998,       !- Starting Z Coordinate
    0.71,                     !- Length
    1.52;                     !- Height

In [9]:
zone_pairs = [(find_matching_zone(i), i.Name) for i in original_subsurfaces]
zone_pairs

[('Block 04 Storey 0', 'Block 04 Storey 0 Wall 0001 Window'),
 ('Block 04 Storey 0', 'Block 04 Storey 0 Wall 0002 Window'),
 ('Block 00 Storey 0', 'Block 00 Storey 0 Wall 0003 Window'),
 ('Block 01 Storey 0', 'Block 01 Storey 0 Wall 0003 Window'),
 ('Block 00 Storey 0', 'Block 00 Storey 0 Wall 0002 Window'),
 ('Block 01 Storey 0', 'Block 01 Storey 0 Wall 0004 Window'),
 ('Block 03 Storey 0', 'Block 03 Storey 0 Wall 0004 Window'),
 ('Block 01 Storey 0', 'Block 01 Storey 0 Wall 0001_1 Door'),
 ('Block 01 Storey 0', 'Block 01 Storey 0 Wall 0002_1 Door'),
 ('Block 02 Storey 0', 'Block 02 Storey 0 Wall 0001_2 Door'),
 ('Block 02 Storey 0', 'Block 02 Storey 0 Wall 0001_1 Door'),
 ('Block 02 Storey 0', 'Block 02 Storey 0 Wall 0002_2 Door'),
 ('Block 06 Storey 0', 'Block 06 Storey 0 Wall 0002_1 Door'),
 ('Block 03 Storey 0', 'Block 03 Storey 0 Wall 0002_2 Door')]

In [25]:
d = {}

for zone, subsurf in zone_pairs:
    if zone not in d.keys():
        d[zone] = []
    d[zone].append(subsurf)


In [28]:
# filter zones w less than one subsurface
afn_zones = {k:v for k,v in d.items() if len(v) >= 2}
afn_zones


{'Block 04 Storey 0': ['Block 04 Storey 0 Wall 0001 Window',
  'Block 04 Storey 0 Wall 0002 Window'],
 'Block 00 Storey 0': ['Block 00 Storey 0 Wall 0003 Window',
  'Block 00 Storey 0 Wall 0002 Window'],
 'Block 01 Storey 0': ['Block 01 Storey 0 Wall 0003 Window',
  'Block 01 Storey 0 Wall 0004 Window',
  'Block 01 Storey 0 Wall 0001_1 Door',
  'Block 01 Storey 0 Wall 0002_1 Door'],
 'Block 03 Storey 0': ['Block 03 Storey 0 Wall 0004 Window',
  'Block 03 Storey 0 Wall 0002_2 Door'],
 'Block 02 Storey 0': ['Block 02 Storey 0 Wall 0001_2 Door',
  'Block 02 Storey 0 Wall 0001_1 Door',
  'Block 02 Storey 0 Wall 0002_2 Door']}

In [26]:
d

{'Block 04 Storey 0': ['Block 04 Storey 0 Wall 0001 Window',
  'Block 04 Storey 0 Wall 0002 Window'],
 'Block 00 Storey 0': ['Block 00 Storey 0 Wall 0003 Window',
  'Block 00 Storey 0 Wall 0002 Window'],
 'Block 01 Storey 0': ['Block 01 Storey 0 Wall 0003 Window',
  'Block 01 Storey 0 Wall 0004 Window',
  'Block 01 Storey 0 Wall 0001_1 Door',
  'Block 01 Storey 0 Wall 0002_1 Door'],
 'Block 03 Storey 0': ['Block 03 Storey 0 Wall 0004 Window',
  'Block 03 Storey 0 Wall 0002_2 Door'],
 'Block 02 Storey 0': ['Block 02 Storey 0 Wall 0001_2 Door',
  'Block 02 Storey 0 Wall 0001_1 Door',
  'Block 02 Storey 0 Wall 0002_2 Door'],
 'Block 06 Storey 0': ['Block 06 Storey 0 Wall 0002_1 Door']}

In [24]:
sort_and_group_objects_dict(zone_pairs, fx=lambda x: x[0])

(['Block 00 Storey 0',
  'Block 01 Storey 0',
  'Block 02 Storey 0',
  'Block 03 Storey 0',
  'Block 04 Storey 0',
  'Block 06 Storey 0'],
 [[('Block 00 Storey 0', 'Block 00 Storey 0 Wall 0003 Window'),
   ('Block 00 Storey 0', 'Block 00 Storey 0 Wall 0002 Window')],
  [('Block 01 Storey 0', 'Block 01 Storey 0 Wall 0003 Window'),
   ('Block 01 Storey 0', 'Block 01 Storey 0 Wall 0004 Window'),
   ('Block 01 Storey 0', 'Block 01 Storey 0 Wall 0001_1 Door'),
   ('Block 01 Storey 0', 'Block 01 Storey 0 Wall 0002_1 Door')],
  [('Block 02 Storey 0', 'Block 02 Storey 0 Wall 0001_2 Door'),
   ('Block 02 Storey 0', 'Block 02 Storey 0 Wall 0001_1 Door'),
   ('Block 02 Storey 0', 'Block 02 Storey 0 Wall 0002_2 Door')],
  [('Block 03 Storey 0', 'Block 03 Storey 0 Wall 0004 Window'),
   ('Block 03 Storey 0', 'Block 03 Storey 0 Wall 0002_2 Door')],
  [('Block 04 Storey 0', 'Block 04 Storey 0 Wall 0001 Window'),
   ('Block 04 Storey 0', 'Block 04 Storey 0 Wall 0002 Window')],
  [('Block 06 Storey 0',